In [1]:
%load_ext autoreload
%autoreload 2

import particle as pt
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mplhep
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import matplotlib.mlab as mlab
import time
from matplotlib.colors import LogNorm
import uproot.models
mplhep.style.use(mplhep.style.ROOT)
from pprint import pprint
from tqdm import tqdm
import polars as pl
import sys
import os
import particle
# using getlogin() returning username
user_name = os.getlogin()

sys.path.append(f'/home/{user_name}/Documents/Atmos_Pandora/apc_atmo_repo/Anatree/')
sys.path.append(f'/home/henrique/Documents/Atmos_Pandora/apc_atmo_repo/personal/Henrique/Analysis/pida/')
from anatree_class import Anatree
from PIDA_class import PIDA
from  ana_tools import *
import plotly.graph_objects as go


plt.rcParams.update({'font.size': 23,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6],
                     })



In [2]:
def loaddata(path:str, entry_stop=None, forceit=False):
    parquet_path= path.replace(".root", ".parquet")
    if os.path.isfile(parquet_path) and not forceit:
        print(path, 'loaded parquet')
        df = pl.read_parquet(parquet_path)
        return df
    pid = PIDA(path, entry_stop=entry_stop)
    dfs:pl.DataFrame
    dfv:pl.DataFrame
    df:pl.DataFrame

    dfs = pid.dfsingles
    dfv = pid.dfvector
    dfp = pid.dfplanes

    df = dfs.join(dfv, on=selection_events())
    df = df.join(dfp, on=selection_events())
    df.write_parquet(file=parquet_path)
    return df
    
df = loaddata("data/pida_shw.root", entry_stop=None, forceit=False)
dfshort = loaddata("data/pida_short.root", entry_stop=None, forceit=False)


data/pida_shw.root loaded parquet
data/pida_short.root loaded parquet


In [ ]:
def getbestof(df:pl.DataFrame ) -> pl.DataFrame:
    allkeys = df.columns
    allkeys = [ k.strip("_U") for k in allkeys if "_U" in k]
    for k in allkeys:
        vartomax = "trknhits_planes"
        if "trkpid" in k:
            vartomax='trkpidndf'
        if k.startswith('allcalo') or k.startswith("trkcalo"):
            vartomax='nallhits_planes'
        
        df = df.with_columns(
            pl.when(
                (pl.col(f'{vartomax}_W') >= pl.col(f'{vartomax}_V')) & 
                (pl.col(f'{vartomax}_W') >= pl.col(f'{vartomax}_U'))  
                ).then(
                    pl.col(f'{k}_W')
                ).otherwise(
                    pl.when(
                        (pl.col(f'{vartomax}_V') >= pl.col(f'{vartomax}_U'))  
                    ).then(
                        pl.col(f'{k}_V')
                    ).otherwise(
                        pl.col(f'{k}_U')
                    )
                ).alias(f"{k}_B")
        )
    return df

dftrkg4 = getbestof(df)
dftrkg4short = getbestof(dfshort)
dftrkg4 = dftrkg4.join(dftrkg4short.group_by(selection_events()).agg(), on=selection_events(), )
dftrkg4 = dftrkg4.join(dftrkg4short.select(selection_events('trkId', 'trkpidpida_B')), on=selection_events('trkId'), how='left', coalesce=True)
dftrkg4 = dftrkg4.with_columns(
    pl.col('trkpidpida_B_right').fill_null(-1).alias('trksmallpida')
).drop('trkpidpida_B_right')
# dftrkg4 

In [ ]:
def ismu():
    return pl.col('trkg4pdg_planes_B').abs()==13

dfall = dftrkg4.sort('trklen').filter(
    pl.col('ccnc_truth') == 0,
    pl.col('nuPDG_truth').abs() == 14
)
dfall = dfall.with_columns(
    pnc = pl.col('trkpurity_planes_B')*pl.col('trkcompleteness_planes_B'),
)


minP = 0.8
minC = 0.8
minV=minP*minC
if 'trkg4mother_planes_B' in dfall.columns:
    dfall_enough_pnc = dfall.group_by(selection_events()).agg(
        ((pl.col('trkg4pdg_planes_B').abs() == 13) & (pl.col('trkg4mother_planes_B')==0) & (pl.col('pnc') > minV) & (pl.col('trklen')>0)).any().alias('hasmu')
    )
else:
    dfall_enough_pnc = dfall.group_by(selection_events()).agg(
        ((pl.col('trkg4pdg_planes_B').abs() == 13) & (pl.col('pnc') > minV) & (pl.col('trklen')>0)).any().alias('hasmu')
    )
print(len(dfall_enough_pnc), len(dfall_enough_pnc.filter(~pl.col('hasmu'))), len(dfall_enough_pnc.filter(~pl.col('hasmu')))/len(dfall_enough_pnc))

dfall_enough_pnc = dfall_enough_pnc.filter(
    pl.col('hasmu')
).select(selection_events())

dfall = dfall.join(dfall_enough_pnc, on=selection_events()).sort(selection_events())
dfallmu = dfall.group_by(selection_events(), maintain_order=True).agg(
    pl.all().last()
)

234571 31490 0.1342450686572509


In [ ]:
# I tried 1e3 cut, virtually no change...
def create_mucandidates(df):
    dfselected = df.sort('trklen').filter(
        pl.col('trkPFPIsTrack'),
        pl.col('trklen')>0
    ).group_by(selection_events(), maintain_order=True).agg(
        pl.col('trkId').last(),
    ).with_columns(
        pl.lit(True).alias("selected_mu")
    )
    df = df.join(dfselected, on=selection_events('trkId'), how='left', coalesce=True).with_columns(
        pl.col('selected_mu').fill_null(False)
    ).group_by(selection_events(), maintain_order=True).agg(
        pl.len().alias('noptions'),
        pl.all()
    ).explode(
        pl.all().exclude(selection_events('noptions'))
    )
    return df

def get_options(df:pl.DataFrame):
    df = df.drop('noptions')
    df = df.group_by(selection_events(), maintain_order=True).agg(
        pl.len().alias('noptions'),
        pl.all()
    ).explode(
        pl.all().exclude(selection_events('noptions'))
    )
    return df
    
def get_biggest(df:pl.DataFrame,
                tailv:dict={'trklen':1,
                       'trkmomllhd':2,
                       'trkPFPScoreIsTrack':2,
                      }
                ):
    listsort = [ k for k in  tailv.keys()]
    listvars = [ f'big_{k}' for k in  tailv.keys()]
    df = df.drop(listvars)
    for var, varsort, tt in zip(listvars, listsort, tailv.values()):
        if varsort not in df.columns:
            continue
        dfsel = df.sort(varsort,).group_by(selection_events(), maintain_order=True).agg(
            pl.col('trkId').tail(tt),
        ).explode(
            'trkId'
        ).with_columns(
            pl.lit(True).alias(var)
        )
        df = df.join(dfsel, on=selection_events('trkId'), how='left', coalesce=True).with_columns(
            pl.col(var).fill_null(False)
        )
    return df
def get_rest(df:pl.DataFrame, restpidcut = 13):
    
    df = get_options(df)
    df = df.filter(~((pl.col('trkpidpida_B')>=restpidcut) & (pl.col('noptions')>1)))
    df = get_options(df)
    firsttails = {
       'trklen': 1,
       'trkmomllhd': 2,
       'trkPFPScoreIsTrack': 2,
    }
    df = get_biggest(df, firsttails)


    dffirst = df.with_columns(
        pl.when(((pl.col('big_trklen')) & (pl.col('big_trkPFPScoreIsTrack')) )).then(True).otherwise(False).alias('selected_mu')
    ).filter(
        pl.col('selected_mu')
    ).sort('trklen').group_by(selection_events(), maintain_order=True).agg(
        pl.col('trkId').last()
    )

    secondtails = {
       'trklen': 1,
       'trkmomllhd': 2,
       'trkPFPScoreIsTrack': 2,
    }
    df = df.join(dffirst, on=selection_events(), how='anti')

    df = get_biggest(df, secondtails)
    df = get_options(df)

    dfsecond = df.with_columns(
        pl.when(((pl.col('big_trklen')) & pl.col('big_trkPFPScoreIsTrack')  )).then(True).otherwise(False).alias('selected_mu')
    ).filter(
        pl.col('selected_mu')
    ).sort('trklen').group_by(selection_events(), maintain_order=True).agg(
        pl.col('trkId').last()
    )

    dfselected = pl.concat([dffirst, dfsecond], how='vertical')
    
    return dfselected

def getfineselection(dforiginal:pl.DataFrame,
                     firstselect={"trklen":1,
                                  "trkmomllhd":3,
                                  "trkPFPScoreIsTrack":3},
                     ):

    df = dforiginal.filter(
        (pl.col('trklen')>10),
    )

    df = get_options(df)
    df = df.filter(
        ~((pl.col('trkmomllhd') > 4) & (pl.col('trklen')<50) & pl.col('noptions')>=2)
    )
    df = get_options(df)
    df = df.filter(
        (pl.col('trkmomllhd')> 2.1*1e-3*pl.col('trklen')+0.005) | (pl.col('noptions')==1),
    )
    df = get_biggest(df,firstselect)
    dfselected = df.with_columns(
        pl.when(((pl.col('big_trklen')) & (pl.col('big_trkmomllhd')) & (pl.col('trkmomllhd')>0) & (pl.col('big_trkPFPScoreIsTrack')))).then(True).otherwise(False).alias('selected_mu')
    ).filter(
        pl.col('selected_mu')
    ).sort('trklen').group_by(selection_events(), maintain_order=True).agg(
        pl.all().last()
    )

    dfselected = dfselected.select(selection_events(['trkId']))
    
    return dfselected

def create_mucandidates_2(df:pl.DataFrame,
                          cut=1.7e3,
                          pidacut=13,
                          pidacut_low=10,
                          restpidcut=13,
                          filter_shower=True,
                          calocut=0, # was 3
                          calocutpida=0., # was 0.1
                          calocut_new=1, 
                          fineselection=True,
                          firstselect={"trklen":1,
                                      "trkmomllhd":3,
                                      "trkPFPScoreIsTrack":3},
                          afterfine = True,
                          includeall = True,
                          ):

    calovariable = "allcalo_planes_B"
    dforiginal = df
    df = df.filter(
        pl.col('trklen')>0
    )

    df = get_options(df)
    df = df.filter(
        ~((pl.col('trklen') >= cut) & (pl.col('noptions')>1)),
        # pl.col('trkPFPIsTrack'),
    )
    df = get_options(df)
    df = df.filter(
        ~((pl.col(calovariable) >= calocutpida) & (pl.col('trkpidpida_B') > pidacut) & (pl.col('noptions')>1)),
        # pl.col('trkPFPIsTrack'),
    )
    if filter_shower: 
    #     # Tested: there is only ~4% of tracks tagged as muon here with PxC > 0.36
        df = get_options(df)
        df = df.filter(
        ~((pl.col(calovariable).is_between(0.3,2)) & ~(pl.col('trkPFPIsTrack')) & (pl.col('noptions')>1))
        )
        df = get_options(df)
        df = df.filter(
        ~((pl.col('trkpidpida_B') < 5) & ~(pl.col('trkPFPIsTrack')) & (pl.col('noptions')>1))# & (pl.col("allcalo")>2))
        )

    df = get_options(df)
    dfallcontained = df.filter(
        (pl.col(calovariable) >= calocut) & ~(pl.col('trkIsContained')) & (pl.col('noptions')>1)
    ).group_by(selection_events()).agg(
    ).with_columns(
        pl.lit(False).alias('allcontained')
    )
    df = df.join(dfallcontained, on=selection_events(), how='left', coalesce=True).with_columns(
        pl.col('allcontained').fill_null(True)
    )
    df = df.filter(
        ~((pl.col(calovariable) >= calocut_new) & (pl.col('trkIsContained')) & ~(pl.col('allcontained'))),
    )
    df = get_options(df)
    df = df.filter(
        ~((pl.col(calovariable) >= calocut) & ~(pl.col('trkPFPIsTrack')) & (pl.col('noptions')>=2)),
        ~((pl.col(calovariable) >= calocut) & ~(pl.col('trkPFPIsTrack')) & (pl.col('trkpidpida_B')>=pidacut_low)),
        ~((pl.col(calovariable) >= calocut) & (pl.col('trkPFPIsTrack')) & (pl.col('noptions')>=2) & (pl.col('trkpidpida_B')>pidacut_low)),
    )
    df = get_options(df)
    # df = df.filter(
    #     ~((pl.col(calovariable) >= calocut) & (pl.col('trkPFPIsTrack')) & (pl.col('noptions')>=2) & (pl.col('trkpidpida_B')>pidacut_low)),
    # )
    df = df.with_columns(
        pl.lit(True).alias('candidate')
    )
    dfcandidates = df
    df = get_options(df)
    if fineselection:
        dfselected = getfineselection(df, firstselect=firstselect) 
    else:
        dfselected = df.sort(
            'trklen',
        ).group_by(selection_events(), maintain_order=True).agg(
            pl.col('trkId').last(),
        )

    if afterfine:
        dfrest = df.join(dfselected, on=selection_events(), how='anti')
        dfrest = dfrest.filter(pl.col('candidate'))
        dfrestselected = get_rest(dfrest, restpidcut=restpidcut)
        dfrestselected = dfrestselected
        dfselected = pl.concat([dfselected, dfrestselected], how='vertical')

    dfrest = dforiginal.join(dfselected, on=selection_events(), how='anti')
    if includeall:
        dfrest = get_options(dfrest)
        dfrest = dfrest.filter(
            ~((pl.col('trklen') >= cut) & (pl.col('noptions')>1)),
        )
        dfrestselected = dfrest.sort('trklen').group_by(selection_events()).agg(
            pl.col('trkId').last()
        )
        dfselected = pl.concat([dfselected, dfrestselected], how='vertical')


    dfselected = dfselected.with_columns(
        pl.lit(True).alias("candidate"),
        pl.lit(True).alias("selected_mu"),
    )
    # dforiginal = df
    dforiginal = dforiginal.join(dfselected, on=selection_events('trkId'), how='left', coalesce=True).with_columns(
        pl.col('candidate').fill_null(False),
        pl.col('selected_mu').fill_null(False),
    )
    dfcandidates = dfcandidates.with_columns(
        change_status=pl.col('candidate')
    ).select(selection_events('trkId','change_status'))
    dforiginal = dforiginal.join(dfcandidates, on=selection_events('trkId'), how='left', coalesce=True)
    dforiginal = dforiginal.with_columns(
        candidate = pl.when(pl.col('change_status')).then(True).otherwise(pl.col('candidate'))
    ).drop('change_status')

    return dforiginal

In [ ]:
def isproton():
    return pl.col('trkg4pdg_planes_B').abs()==2212
def isparticle(pdg):
    return pl.col('trkg4pdg_planes_B').abs()==pdg
def onlyone(n:int=1):
    return pl.col('noptions')==n
def morethan(n:int=2):
    return pl.col('noptions')>=n

def get_all_the_rest(dffull:pl.DataFrame, dfcutted:pl.DataFrame):
    return dffull.join(dfcutted.group_by(selection_events()).agg(), on=selection_events(), how='anti')
def create_proton_candidate(df:pl.DataFrame,
                            cut_pida_easy=10,
                            cut_pida_easy_shower=13,
                            maxmom=1.5,
                            maxcalo=0.8,
                            apply_shower_filter=False,
                            nopt_shower_filter=3,
                            ):
    df = df.filter(
        ~pl.col('selected_mu'),
        pl.col('trklen')>0,
    )

    df = get_options(df)
    df = df.filter(
        ((pl.col('trkpidpida_B')>cut_pida_easy) & (pl.col('trkPFPIsTrack'))) |
        ((pl.col('trkpidpida_B')>cut_pida_easy_shower) & ~(pl.col('trkPFPIsTrack')))
        # | ((onlyone()) & (pl.col('trkpidpida_B')>6) & (pl.col('trkmomrange_pr')<0.6) & (pl.col('trkPFPIsTrack')))
    ).filter(
        pl.col('trkcalo_planes_B')<maxcalo,
        pl.col('trkmomrange_pr').is_between(0,maxmom)
    )
    if apply_shower_filter:
        df = df.filter(
            ~(~(pl.col('trkPFPIsTrack')) & morethan(nopt_shower_filter))
        )
    df = get_options(df)
    df = df.with_columns(
        pl.lit(True).alias('selected_pr')
    )
    return df



In [ ]:
dfsimple = create_mucandidates(dfall)
dfmus = create_mucandidates_2(dfall)
dfprselected = create_proton_candidate(dfmus)
dfprselected = dfmus.join(dfprselected.select(selection_events('trkId', 'selected_pr')), on=selection_events('trkId'), how='left', coalesce=True)
dfprselected = dfprselected.with_columns(
    pl.col('selected_pr').fill_null(False)
)

In [ ]:
def create_pion_candidate(df:pl.DataFrame,
                          minimum_options_after_remove_mu=2,
                          return_full_efficiency=False,
                          do_not_apply_small_cut_pida=False,
                          do_not_apply_cut_en=False,
                          ):
    df = df.filter(
        pl.col('trklen')>0,
    )
    df = df.filter(
        ~pl.col('selected_mu'),
    )
    df = get_options(df)
    dfwithpr = df.filter(pl.col('selected_pr')).group_by(selection_events()).agg(
        pl.col('selected_pr').count().alias('npr'),
    )
    df = df.join(dfwithpr, on=selection_events(), how='left', coalesce=True)
    df = df.with_columns(
        pl.col('npr').fill_null(0)
    )
    if return_full_efficiency:
        df = df.filter(
            ~pl.col('selected_pr'),
        )
        df = get_options(df)
        df = df.with_columns(
            pl.lit(True).alias('selected_pi')
        )
        return df

    df = df.filter(
        ~pl.col('selected_pr'),
        morethan(minimum_options_after_remove_mu), # Could not find any reliable way if there is muon and no proton
        pl.col('trkPFPIsTrack')
    )

    df = df.filter(
        (do_not_apply_small_cut_pida) | ((pl.col('trksmallpida').is_between(1.,5)) | (pl.col('trksmallpida').is_between(7.5,10))),
        (do_not_apply_cut_en) | (pl.col('trkmomrange_mu').is_between(0.15, 0.5) & pl.col('trkcalo_planes_B').is_between(0.05, 0.5)),
        # pl.col('npr')>=1,
        pl.col('trkIsContained')
    )
    df = get_options(df)
    df = df.with_columns(
        pl.lit(True).alias('selected_pi')
    )

    return df

In [ ]:
# dfprselected = create_proton_candidate(dfmus)
dfprpiselected = create_pion_candidate(dfprselected,
                                       minimum_options_after_remove_mu=2,
                                    #    do_not_apply_cut_en=True,
                                    #    do_not_apply_small_cut_pida=True,
                                       )
dfprpiselected = dfprselected.join(dfprpiselected.select(selection_events('trkId', 'selected_pi')), on=selection_events('trkId'), how='left', coalesce=True)
dfprpiselected = dfprpiselected.with_columns(
    pl.col('selected_pi').fill_null(False)
)

def get_n_prpi(df:pl.DataFrame):
    
    dfwith = df.filter(pl.col('selected_pi')).group_by(selection_events()).agg(
        pl.col('selected_pr').count().alias('npr'),
    )
    df = df.join(dfwith, on=selection_events(), how='left', coalesce=True)
    df = df.with_columns(
        pl.col('npr').fill_null(0)
    )
    dfwith = df.filter(pl.col('selected_pi')).group_by(selection_events()).agg(
        pl.col('selected_pi').count().alias('npi'),
    )
    df = df.join(dfwith, on=selection_events(), how='left', coalesce=True)
    df = df.with_columns(
        pl.col('npi').fill_null(0)
    )
    return df
dfprpiselected = get_n_prpi(dfprpiselected)

In [ ]:
def simple_energy(df:pl.DataFrame):
    df.